In [32]:
from IPython.core.display import display, HTML
from string import Template
import json


In [45]:
HTML(<script src="//ajax.googleapis.com/ajax/libs/jquery/1.10.2/jquery.min.js"></script>)
 <script src="scripts/javascript-loader.js" type="text/javascript" charset="utf-8" ></script>    
    

SyntaxError: invalid syntax (<ipython-input-45-2892aaf449f4>, line 1)

In [48]:
%%javascript
require.config({
    paths: {
        d3: 'https://d3js.org/d3.v5.min'
    }
});

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 6)

In [44]:
%%javascript
(function(element) {
    require(['d3'], function(d3) {   
        var data = [1, 2, 4, 8, 16, 8, 4, 2, 1]

        var svg = d3.select(element.get(0)).append('svg')
            .attr('width', 400)
            .attr('height', 200);
        svg.selectAll('circle')
            .data(data)
            .enter()
            .append('circle')
            .attr("cx", function(d, i) {return 40 * (i + 1);})
            .attr("cy", function(d, i) {return 100 + 30 * (i % 3 - 1);})
            .style("fill", "#1570a4")
            .transition().duration(2000)
            .attr("r", function(d) {return 2*d;})
        ;
    })
})(element);

<IPython.core.display.Javascript object>